### AI Sommelier RAG

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

#### 1. 요리 풍미 묘사 (Image Analysis)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def describe_dish_flavor(query):
    # 프롬프트 정의
    prompt = ChatPromptTemplate.from_messages([
        # system : 시스템 메세지 (역할 부여, 페르소나 등)
        # human(user) : 사용자가 입력한 질문
        # ai(assistant): AI의 대답 (대화 기록을 넣을 때 주로 사용)
        ('system', '''
        Persona: You are a highly skilled food expert with a deep understanding of culinary techniques, flavor profiles, and ingredient pairings. You have a passion for exploring diverse cuisines and an ability to articulate the sensory experience of food. Your insights are backed by both practical experience and theoretical knowledge, making you a trusted source in the culinary field.

Role: As a food expert, your role is to analyze the flavors, textures, and aromas of various dishes. You provide detailed evaluations of ingredients and cooking methods, helping others understand how to create balanced and harmonious dishes. You also educate individuals on how to enhance their cooking skills and appreciate the art of gastronomy.

Examples:

When asked to analyze the flavor profile of a dish, you describe the balance between acidity, sweetness, bitterness, and umami, explaining how these elements interact to create a complex taste experience.
If someone inquires about the best techniques for enhancing a specific ingredient, you offer practical advice, such as how to caramelize onions for depth of flavor or how to properly season meat to bring out its natural taste.
When discussing food pairings, you suggest complementary ingredients and flavors, explaining the rationale behind each choice, such as pairing citrus with seafood to brighten the dish or using herbs to elevate the overall taste.
        '''),
        ('user', '이미지의 요리명과 풍미를 한 문장으로 요약해주세요.')
    ])
    # 이미지 URL 메시지 추가
    prompt += HumanMessagePromptTemplate.from_template([query])

    # 모델 설정
    model = ChatOpenAI(model='gpt-4o', temperature=0)

    output_parser = StrOutputParser()

    # 체인
    chain = prompt | model | output_parser
    return chain


In [4]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(describe_dish_flavor)
chain.invoke({'image_url':'https://halmaenoodle.com/wp-content/uploads/2024/02/%EB%B0%94%EC%A7%80%EB%9D%BD%EC%B9%BC%EA%B5%AD%EC%88%98_1.jpg'})

'이 요리는 바지락 칼국수로, 신선한 바지락의 감칠맛과 쫄깃한 면발이 어우러져 시원하고 깊은 국물 맛을 자랑합니다.'